# 宏观经济数据获取

初始宏观数据获取：https://uestc.feishu.cn/wiki/DrMYw98b2iVGRbkBfH3cGzeTnoe#EdkHdlhkkoahbEx18ZxcszeRnzb

In [7]:
import pandas as pd

In [9]:
macro = pd.read_csv('../数据/macro.csv')
# 去除有空数据的行
macro.dropna(inplace=True)
# date列转换为datetime类型,作为索引
macro['date'] = pd.to_datetime(macro['date'])
macro.set_index('date', inplace=True)
#去除 Unnamed: 0  
macro.drop(columns='Unnamed: 0', inplace=True)
# 去除 'instrument`
macro.drop(columns='instrument', inplace=True)
# 保存数据
macro.to_excel('../数据/macro.xlsx')

# 情绪数据获取

In [6]:
import akshare as ak

index_fear_greed_funddb_df = ak.index_fear_greed_funddb(symbol="沪深300")

# date作为索引
index_fear_greed_funddb_df.set_index('date',inplace=True)
index_fear_greed_funddb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, 2022-04-06 to 2024-04-01
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   fear    484 non-null    float64
 1   index   484 non-null    float64
dtypes: float64(2)
memory usage: 11.3+ KB


In [3]:
index_fear_greed_funddb_df.date

0      2022-04-06
1      2022-04-07
2      2022-04-08
3      2022-04-11
4      2022-04-12
          ...    
479    2024-03-26
480    2024-03-27
481    2024-03-28
482    2024-03-29
483    2024-04-01
Name: date, Length: 484, dtype: object